步骤 1: 环境设置与库导入

In [ ]:
# 导入必要的库
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import requests
import json
import hashlib
import time
from models.mha import TextEncoderWithMHA
from models.dynamic_u_net import UNet

# 提示：确保已经安装了所有依赖库，可通过运行以下命令安装：
# pip install -r requirements.txt

# 检查 GPU 是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用的设备: {device}")

步骤 2: 文本编码

In [ ]:
# 提示：这里使用 TextEncoderWithMHA 对输入的文本进行编码，得到语义嵌入
# 初始化文本编码器
text_encoder = TextEncoderWithMHA()
text = ["dog", "grass"]  # 输入文本
print(f"输入的文本: {text}")

# 对文本进行编码
result = text_encoder.encode_text(text)
semantic_embedding = result["embeddings"].to(device)
print("语义嵌入的形状:", semantic_embedding.shape)

步骤 3: 动态权重计算

In [ ]:
# 提示：调用 /api/v5/weight/calculate 接口计算动态权重
# 构造请求数据
request_data = {
    "base_prompt": "A dog on the grass",
    "attributes": [
        {
            "name": "object",
            "type": "text",
            "value": text[0],
            "initial_weight": 0.8,
            "constraints": {
                "min_weight": 0.4,
                "max_weight": 0.9,
            }
        },
        {
            "name": "background",
            "type": "text",
            "value": text[1],
            "initial_weight": 0.2,
            "constraints": {
                "min_weight": 0.1,
                "max_weight": 0.5,
            }
        }
    ],
    "temperature": 1.8,
    "fallback_strategy": "creative",
}

# 获取时间戳
timestamp = str(int(time.time()))
# 检查时间戳是否在有效范围内（假设有效期为 60 秒）
if int(time.time()) - int(timestamp) > 60:
    print("时间戳已过期，请重新获取")
    timestamp = str(int(time.time()))

data_str = json.dumps(request_data, sort_keys=True)

# 计算签名
message = f"{data_str}{timestamp}"
signature = hashlib.sha256(message.encode()).hexdigest()

headers = {
    "Content-Type": "application/json",
    "X-Api-Key": "api_key",
    "X-Timestamp": timestamp,
    "X-Signature": signature,
}

# 提示：确保 dynamic_weights 服务正在运行，监听在 http://127.0.0.1:5000
# 调用 dynamic_weights 接口
url = 'http://127.0.0.1:5000/api/v5/weight/calculate'
response = requests.post(url, headers=headers, json=request_data)

if response.status_code == 200:
    response_data = response.json()
    if response_data["code"] == 200:
        final_weights = response_data["data"]["final_weights"]
        weights = [final_weights[key] for key in text]
        print("最终权重:", final_weights)
    else:
        print(f"Failed to get dynamic weights: {response_data}")
        weights = [1.0] * len(text)
else:
    print(f"Failed to get dynamic weights: {response.text}")
    weights = [1.0] * len(text)

步骤 4: 图像加载与预处理

In [ ]:
# 提示：确保 image3.jpg 文件存在于当前工作目录中
# 加载图片
image_path = 'image3.jpg'  # 输入图像地址
image = Image.open(image_path).convert('RGB')
print(f"加载的图像: {image_path}")

# 图像预处理
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])
input_image = transform(image).unsqueeze(0).to(device)
print("输入图像的形状:", input_image.shape)

步骤 5: 模型初始化与推理

In [ ]:
# 提示：初始化 UNet 模型进行图像生成
# 初始化 UNet 模型
model = UNet(in_channels=3, out_channels=3).to(device)

# 模型推理
output = model(input_image, semantic_embedding, weights)
output = output.cpu().squeeze(0).detach().numpy()

# 归一化到 [0, 1] 范围
output = (output - output.min()) / (output.max() - output.min() + 1e-8)

# 调整维度以符合 imshow 要求 (H, W, C)
output = output.transpose(1, 2, 0)

步骤 6: 结果可视化

In [ ]:
# 提示：显示和保存生成的图像
# 显示和保存结果
plt.imshow(output)
plt.axis('off')
plt.savefig('output_image.jpg', bbox_inches='tight', pad_inches=0)
plt.show()
print("生成的图像已保存为 output_image.jpg")